<a href="https://colab.research.google.com/github/paulsubarna/CONFEDMADE/blob/main/tutorial_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TUTORIAL 4: REPLAY BUFFERS IN SEQUENTIAL TRAINING

In this notebook, we will explore different replay buffer strategies. The two most common approaches are as follows:
- Gnerative Replay buffer using VAEs- "Store generated samples"
- Experience Replay- "Store real samples from the dataset"

We will try to investigate how does storing real samples and generated samples impact into the sequential training of our model.

#### DATA Preparation and Loading

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms
from tqdm import tqdm
from torchvision.models import resnet18
import random
import matplotlib.pyplot as plt

import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
mean = torch.tensor([0.4589, 0.4384, 0.4011])
std = torch.tensor([0.2793, 0.2724, 0.2835])

# Apply transforms
#transforms = transforms.Compose(
#    [transforms.ToTensor()
#    ])

mnist_trainset = datasets.MNIST(root='/app/src/Mnist', train=True, download=True, transform= transforms.ToTensor())
mnist_testset = datasets.MNIST(root='/app/src/Mnist', train=False, download=True, transform= transforms.ToTensor())

len(mnist_trainset), len(mnist_testset) # 60000, 10000
config = {'size': 28, 'channels': 1, 'classes': 10}

### Spliting dataset into N contexts

- Let us first divide our dataset into five sequential tasks and we will do it based on the labels
- Split with labels such that,  context 1: [0,1], context 2: [2,3],  context 1: [4,5], context 2: [6,7],  context 1: [8,9]

In [ ]:
train_data_cl = []
test_data_cl = []
labels = [[0,1], [2,3], [4,5], [6,7], [8,9]]
labels_ = [[0,1], [0,1,2,3], [0,1,2,3,4,5], [0,1,2,3,4,5,6,7], [0,1,2,3,4,5,6,7,8,9]] ### For test sets, each contexts should have all the classes from the previous contexts
for j in range(5):
    train_data_cl.append([])
    test_data_cl.append([])
    for i in range(len(mnist_trainset)):
        if mnist_trainset[i][1] in labels[j]:
            train_data_cl[j].append((mnist_trainset[i][0], mnist_trainset[i][1]))
    for i in range(len(mnist_testset)):
        if mnist_testset[i][1] in labels_[j]:
            test_data_cl[j].append((mnist_testset[i][0], mnist_testset[i][1]))


#train_data_cl[0][0]

#### Sequential training

Let us now use this set of tasks to train a neural network sequentially, where we define a simple classifier or convolutational networks and train it on all the sequence of tasks  

"" TO-DO ""
- Define a simple MLP network or a Convolutional Network
- Define your training loop
- Train it on all the contexts

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        """ TODO
        Include the rest of the layers here: Must be Linear layers and activation functions
        """

    def forward(self, x):
        #x = nn.Flatten()(x)
        """ TO-DO """

        return x

def train_classifier(model, train_data, test_data, num_epochs=10, batch_size=32, learning_rate=0.001):
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Create data loaders
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            """ TO-DO """

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Evaluation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            """ TO-DO """






    print(f"Accuracy of the model on the test data: {100 * correct / total:.2f}%")


### Define the model
model = Classifier(input_dim=28*28, hidden_dim=256, output_dim=10)
import os
if not os.path.exists('/app/src/model_replay'):
    os.makedirs('/app/src/model_replay')

### Train the model on each context
for i in range(5):

    train_classifier(model, train_data_cl[i], test_data_cl[i], num_epochs=5, batch_size=32, learning_rate=0.1)
    #torch.save(model.state_dict(), f'/app/src/model_replay/replay_{i}.pth')


We could clearly observe the phenomenon of catastrophic forgetting here.
We should also see the same phenomenon if we have to evaluate on individial contexts where the model performs the best on the last task and bad performance on the earlier tasks

In [ ]:
### Evaluate the model on each context
def evaluate_classifier(model, test_data, batch_size=32):
    # Create data loader
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Evaluation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            """ TO-DO """


    print(f"Accuracy of the model on the test data: {100 * correct / total:.2f}%")

for i in range(5):
    evaluate_classifier(model, test_data_cl[i], batch_size=32)

#### Accomodate Replay buffers

We have two options
- Store generative samples using VAEs
- Store real samples as experience replay

But, first, we need to define our VAE model, train it on all the contexts and obderve its efficacy to generate new data

1. Generative Replay Buffers

- We will use VAE to generate new samples, that you will store in the memory buffers
- Define your VAE model
- we will generate samples based on class instances [0], [1], etc
- So, we will train our VAE model on samples from individual class instances. That means it is necessary to split the dataset based on the samples.

##### Split the dataset per labels

In [ ]:
train_data = []
test_data = []
#labels = [[0,1,2], [3,4,5], [6,7,8]]
labels = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

#labels = [[0], [1], [2]]
for j in range(10):
    train_data.append([])
    test_data.append([])
    for i in range(len(mnist_trainset)):
        if mnist_trainset[i][1] in labels[j]:
            train_data[j].append((mnist_trainset[i][0], mnist_trainset[i][1]))
    for i in range(len(mnist_testset)):
        if mnist_testset[i][1] in labels[j]:
            test_data[j].append((mnist_testset[i][0], mnist_testset[i][1]))


train_data[0][0]

##### Variational Autoencoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Encoder, self).__init__()

        """ TO-DO """


    def forward(self, x):
        #x = nn.Flatten()(x)
        """ TO-DO """

        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()

        """ TO-DO """


    def forward(self, z):
        """ TO-DO """

        return z

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, output_dim)
        self.decoder = Decoder(output_dim, hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        """ TO-DO """

        return mu + eps * std

    def forward(self, x):
        """ TO-DO """

        return x_recon, mu, logvar
    def loss_function(self, x, x_recon, mu, logvar):
        """ TO-DO """

        return BCE + 5 * KLD

    def sample(self, z):
        """ TO-DO
        1. Generate samples from the latent space
        2. Use the decoder to reconstruct the images
        3. Return the reconstructed images
        """
        with torch.no_grad():

            return z.view(-1, 1, 28, 28)
    def generate(self, num_samples, output_dim):
        with torch.no_grad():
            z = torch.randn(num_samples, output_dim)
            samples = self.decoder(z)
            return samples.view(-1, 1, 28, 28)

##### Train your VAE Model

In [ ]:
def train(train_data=None, test_data=None, t= None):
    # Hyperparameters
    input_dim = 28 * 28
    hidden_dim = 400
    output_dim = 20
    batch_size = 64
    num_epochs = 5
    learning_rate = 1e-3

    # Data loader
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Model, loss function and optimizer
    model = VAE(input_dim, hidden_dim, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(tqdm(train_loader)):
        """ TO-DO
        1. Define the training loop
        """




        print(f'Epoch {epoch + 1}, Loss: {train_loss / len(train_loader.dataset)}')

        # Save the model checkpoint
        torch.save(model.state_dict(), f'/app/src/vae_mnist_{t}.pth')
    return model


def test(model, test_data = None, batch_size= None, t= None):
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
    # Load the model checkpoint
    model.load_state_dict(torch.load(f'/app/src/vae_mnist_{t}.pth'))
    for epoch in range(1):
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for batch_idx, (data, _) in enumerate(tqdm(test_loader)):
                data = data.view(-1, 28 * 28)
                x_recon, mu, logvar = model(data)
                loss = model.loss_function(data, x_recon, mu, logvar)
                test_loss += loss.item()

        print(f'Test Loss: {test_loss / len(test_loader.dataset)}')

        # Generate samples
        samples = model.generate(64, output_dim=20)
        grid = make_grid(samples, nrow=8)
        plt.imshow(grid.permute(1, 2, 0).numpy())
        plt.axis('off')
        plt.show()



input_dim = 28 * 28
hidden_dim = 400
output_dim = 2
batch_size = 128
num_epochs = 10
learning_rate = 1e-3
for i in range(10):
    model = train(train_data[i], test_data[i], t= i)
    test(model, test_data[i], batch_size= 64, t= i)


#### Populate Replay Buffer with your generated samples

For each of these use cases, we will adopt reservoir sampling. We first fix the size of the memory buffer to N samples, for instance 10000. The idea is that at each time step, we should save equal number of samples for each of the previously seen class instances.

If you recall, each of our contexts has only 2 class labels.
So, at the end of the first task, we will store 5000 samples from labels [0] and [1].
Similarily at the end of the second task, we will store 2500 samples from labels [0], [1], [2], [3]
Continue it till the final context

Replay buffer for each context should look this:
- context_0: [5000, 5000]
- context_1: [2500, 2500, 2500, 2500]
- context_2:[1666, 1666, 1666, 1666, 1666, 1666]
- context_3: [1250, 1250, 1250, 1250, 1250, 1250, 1250, 1250]




In [ ]:
generator = VAE(28*28, 400, 20)
mem_size = None
contexts = 5
replay = []
for i in range(contexts - 1):
""" TO-DO

1. Generate samples from the pretrained VAE for each labels
2. Store the generated samples in the replay buffer
"""



In [ ]:
model = Classifier(input_dim=28*28, hidden_dim=256, output_dim=10)
for i in range(5):
    if i == 0:
        train_classifier(model, train_data_cl[i], test_data_cl[i], num_epochs=5, batch_size=64, learning_rate=0.1)
    else:
        train_classifier(model, train_data_cl[i]+ replay[i-1], test_data_cl[i], num_epochs=10, batch_size=64, learning_rate=0.1)
    #torch.save(model.state_dict(), f'/app/src/model_replay/replay_{i}.pth')

#### Experience Replay

Now, we could perform the similar experiments with real data samples directly from the dataset. Use the same reservoir sampling approach to populate the replay buffer

In [ ]:
mem_size = 10000
labels = [[[0],[1]], [[0],[1],[2],[3]], [[0],[1],[2],[3],[4],[5]], [[0],[1],[2],[3],[4],[5],[6],[7]], [[0],[1],[2],[3],[4],[5],[6], [7],[8],[9]]]
""" TO_DO """

#### Train all the contexts using replay buffer

In [ ]:
model = Classifier(input_dim=28*28, hidden_dim=256, output_dim=10)
for i in range(5):

    #model.load_state_dict(torch.load(f'/app/src/model_replay/replay_{i}.pth'))
    #model = train_classifier(model, train_data[i], test_data[i], num_epochs=5, batch_size=32, learning_rate=0.001)
    if i == 0:
        train_classifier(model, train_data_cl[i], test_data_cl[i], num_epochs=5, batch_size=64, learning_rate=0.1)
    else:
        train_classifier(model, train_data_cl[i]+ replay[f'context_{i-1}'], test_data_cl[i], num_epochs=10, batch_size=64, learning_rate=0.1)
    #torch.save(model.state_dict(), f'/app/src/model_replay/replay_{i}.pth')